In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import math
from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import datetime
from keras.regularizers import l2
from keras.models import load_model

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns


In [5]:
import sys
import time
import zlib

In [6]:
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix

In [7]:
import warnings
warnings.filterwarnings('once')

Digital DNA
Model account behaviour by defining the following alphabet, of cardinality N = 3

$B^3_{type} = {\{A, C, T\}}$

A $\leftarrow$ tweet,

C $\leftarrow$ reply,

T $\leftarrow$ retweet

In [8]:
### Users

# Genuine accounts.
gen = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/genuine_accounts.csv/users.csv')

# Social spambots.
ss1 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_1.csv/users.csv')
ss2 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_2.csv/users.csv')
ss3 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_3.csv/users.csv')

# Traditional spambots.
ts1 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_1.csv/users.csv')
ts2 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_2.csv/users.csv')
ts3 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_3.csv/users.csv')
ts4 = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_4.csv/users.csv')

In [20]:
### Tweets

# Genuine accounts.
# gen_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/genuine_accounts.csv/tweets.csv')

# Social spambots.
ss1_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_1.csv/tweets.csv')
ss2_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_2.csv/tweets.csv')
ss3_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_3.csv/tweets.csv')

# Traditional spambots.
ts1_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_1.csv/tweets.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
# gen_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/mini_datasets/genuine.csv', nrows = 1000)
gen_tweets = pd.read_csv('/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/genuine_accounts.csv/tweets.csv', usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25], skiprows = 500, nrows = 200000)
print(gen_tweets.count)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<bound method DataFrame.count of         582077352907128832  ... 2015-05-01 12:57:19.1
0       582073617896239105  ...   2015-05-01 12:57:19
1       582060608486412288  ...   2015-05-01 12:57:19
2       582060470514794496  ...   2015-05-01 12:57:19
3       582055238988746752  ...   2015-05-01 12:57:19
4       582054859685302272  ...   2015-05-01 12:57:19
...                    ...  ...                   ...
199995  561185729972170752  ...   2015-05-01 13:07:51
199996  515263353380671488  ...   2015-05-01 13:07:51
199997  561185648451670016  ...   2015-05-01 13:07:51
199998  515229340821635074  ...   2015-05-01 13:07:51
199999  561185542591619073  ...   2015-05-01 13:07:51

[200000 rows x 26 columns]>


In [65]:
gen_tweets.tail()

,582077352907128832,RT @cbcasithappens: Scott Bonner wins Lemony Snicket prize for keeping @fergusonlibrary open during unrest. Our November inv w/ Scott: http…,"<a href=\http://twitter.com\"" rel=\""nofollow\"">Twitter Web Client</a>""",678033,Unnamed: 4,0,0.1,Unnamed: 7,581572022662090752,\N,\N.1,Unnamed: 11,Unnamed: 12,7,0.2,0.3,Unnamed: 16,Unnamed: 17,Unnamed: 18,0.4,1,2,Sun Mar 29 07:10:30 +0000 2015,2015-03-29 09:10:30,2015-05-01 12:57:19,2015-05-01 12:57:19.1
199995,561185729972170752,@camacaron @_hagau @steamedmantou @bbbro_omomu...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",152194153,NaN,5.61185e+17,9.50734e+08,camacaron,0,\N,\N,NaN,NaN,0,0,0,NaN,NaN,\N,0,0,4,Fri Jan 30 15:34:39 +0000 2015,2015-01-30 16:34:39,2015-05-01 13:07:51,2015-05-01 13:07:51
199996,515263353380671488,@harle_beth to see meeee,"<a href=\http://twitter.com/download/iphone\"" ...",1683489438,NaN,5.14888e+17,2.194e+09,harle_beth,0,\N,\N,NaN,NaN,0,0,1,NaN,NaN,\N,0,0,1,Thu Sep 25 22:15:31 +0000 2014,2014-09-26 00:15:31,2015-05-01 13:07:51,2015-05-01 13:07:51
199997,561185648451670016,"RT @GeorgeTakei: Don't be a pussy, take it lik...","<a href=\http://twitter.com\"" rel=\""nofollow\""...",152194153,NaN,0,0,NaN,561185524330033154,\N,\N,NaN,NaN,489,0,0,NaN,NaN,NaN,0,0,1,Fri Jan 30 15:34:19 +0000 2015,2015-01-30 16:34:19,2015-05-01 13:07:51,2015-05-01 13:07:51
199998,515229340821635074,RT @corawest_: I need to sort my laugh out,"<a href=\http://twitter.com/download/iphone\"" ...",1683489438,NaN,0,0,NaN,515219090072698880,\N,\N,NaN,NaN,6,0,0,NaN,NaN,\N,0,0,1,Thu Sep 25 20:00:22 +0000 2014,2014-09-25 22:00:22,2015-05-01 13:07:51,2015-05-01 13:07:51
199999,561185542591619073,this is what they mean when they say life is g...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",152194153,NaN,0,0,NaN,0,\N,\N,NaN,NaN,0,0,0,NaN,NaN,\N,0,0,0,Fri Jan 30 15:33:54 +0000 2015,2015-01-30 16:33:54,2015-05-01 13:07:51,2015-05-01 13:07:51


In [60]:
gen_tweets.columns = ["id","text","source","user_id","truncated","in_reply_to_status_id","in_reply_to_user_id","in_reply_to_screen_name","retweeted_status_id","drop1","geo","place","contributors","retweet_count","reply_count","favorite_count","favorited","retweeted","possibly_sensitive","num_hashtags","num_urls","num_mentions","created_at","timestamp","crawled_at","updated",]

In [61]:
gen_tweets.drop(['drop1'], axis=1)

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,582073617896239105,RT @allisonkilkenny: Do watch @LastWeekTonight...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,580108730014031872,\N,NaN,NaN,53,0.0,0.0,NaN,NaN,NaN,1,1,2,Sun Mar 29 06:55:39 +0000 2015,2015-03-29 08:55:39,2015-05-01 12:57:19,2015-05-01 12:57:19
1,582060608486412288,Thinking about thinning my twitter feed down t...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,0,\N,NaN,NaN,0,0.0,1.0,NaN,NaN,\N,0,0,1,Sun Mar 29 06:03:57 +0000 2015,2015-03-29 08:03:57,2015-05-01 12:57:19,2015-05-01 12:57:19
2,582060470514794496,"RT @Sherman_Alexie: In conservative parlance, ...","<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,573588291330535424,\N,NaN,NaN,118,0.0,0.0,NaN,NaN,\N,0,0,1,Sun Mar 29 06:03:24 +0000 2015,2015-03-29 08:03:24,2015-05-01 12:57:19,2015-05-01 12:57:19
3,582055238988746752,RT @CBR: EXCLUSIVE: @marikotamaki Casts #TMNT'...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,578053226185920512,\N,NaN,NaN,7,0.0,0.0,NaN,NaN,NaN,1,1,2,Sun Mar 29 05:42:37 +0000 2015,2015-03-29 07:42:37,2015-05-01 12:57:19,2015-05-01 12:57:19
4,582054859685302272,RT @nytimes: Harper Lee's new novel now has a ...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,580832825240350722,\N,NaN,NaN,170,0.0,0.0,NaN,NaN,NaN,0,1,1,Sun Mar 29 05:41:07 +0000 2015,2015-03-29 07:41:07,2015-05-01 12:57:19,2015-05-01 12:57:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,541633313660612608,@tomcoates oops http://t.co/nh0k6aJcEY @daniel...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,541632519733407744,12514,tomcoates,0,\N,NaN,NaN,0,0.0,1.0,NaN,NaN,NaN,0,1,4,Sun Dec 07 16:40:19 +0000 2014,2014-12-07 17:40:19,2015-05-01 12:57:22,2015-05-01 12:57:22
1996,541633271214649344,@tomcoates I think this ep speaks volumes abou...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,541632519733407744,12514,tomcoates,0,\N,NaN,NaN,0,0.0,0.0,NaN,NaN,\N,0,0,4,Sun Dec 07 16:40:09 +0000 2014,2014-12-07 17:40:09,2015-05-01 12:57:22,2015-05-01 12:57:22
1997,541632363705282562,@tomcoates that was 1996. Queers had to come o...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,541630985406644224,12514,tomcoates,0,\N,NaN,NaN,0,0.0,0.0,NaN,NaN,\N,0,0,4,Sun Dec 07 16:36:33 +0000 2014,2014-12-07 17:36:33,2015-05-01 12:57:22,2015-05-01 12:57:22
1998,541631958405500930,@tomcoates to see the change look at the clip ...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,541630985406644224,12514,tomcoates,0,\N,NaN,NaN,0,0.0,0.0,NaN,NaN,\N,0,0,4,Sun Dec 07 16:34:56 +0000 2014,2014-12-07 17:34:56,2015-05-01 12:57:22,2015-05-01 12:57:22


In [33]:
ss1_tweets.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,532627591686275072,I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:14:48 +0000 2014,2014-11-12 21:14:48,2014-11-12 21:44:09,2014-11-12 21:44:09
1,532624255058706432,http://t.co/HyI5EQKz6Q,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:01:32 +0000 2014,2014-11-12 21:01:32,2014-11-12 21:44:09,2014-11-12 21:44:09
2,532513524460052480,"Tutti a tavola, con il filetto di baccalà. htt...","<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 12:41:32 +0000 2014,2014-11-12 13:41:32,2014-11-12 21:44:09,2014-11-12 21:44:09
3,532297646669852672,http://t.co/NAHQ4l2pUy,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:23:43 +0000 2014,2014-11-11 23:23:43,2014-11-12 21:44:09,2014-11-12 21:44:09
4,532295960807100416,Gold - Spandau Ballet http://t.co/o8ZJHt7Neu,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:17:01 +0000 2014,2014-11-11 23:17:01,2014-11-12 21:44:09,2014-11-12 21:44:09


In [56]:
gen_tweets.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,drop1,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,drop2
0,593750427792867329,RT @TheAtlanticPOL: Watch Baltimore native @ta...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,0.000000e+00,0,NaN,5.937349e+17,\N,\N,NaN,NaN,231.0,0,0,NaN,NaN,NaN,0.0,1,3,Thu Apr 30 12:15:07 +0000 2015,2015-04-30 14:15:07,2015-05-01 12:57:19,2015-05-01 12:57:19,NaN
1,593742580774371329,@thelancearthur @hotdogsladies I didn't realiz...,"<a href=\http://tapbots.com/tweetbot\"" rel=\""n...",678033,NaN,5.936419e+17,143569237,thelancearthur,0.000000e+00,\N,\N,NaN,NaN,0.0,0,0,NaN,NaN,\N,0.0,0,2,Thu Apr 30 11:43:57 +0000 2015,2015-04-30 13:43:57,2015-05-01 12:57:19,2015-05-01 12:57:19,NaN
2,593659318479687680,RT @sippey: Here's my one paragraph review of ...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0.000000e+00,0,NaN,5.934603e+17,\N,\N,NaN,NaN,11.0,0,0,NaN,NaN,NaN,0.0,1,1,Thu Apr 30 06:13:05 +0000 2015,2015-04-30 08:13:05,2015-05-01 12:57:19,2015-05-01 12:57:19,NaN
3,593654865689124865,@wkamaubell I stand by you. I don't want men t...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,5.936491e+17,19119809,wkamaubell,0.000000e+00,\N,\N,NaN,NaN,0.0,0,2,NaN,NaN,\N,0.0,0,1,Thu Apr 30 05:55:24 +0000 2015,2015-04-30 07:55:24,2015-05-01 12:57:19,2015-05-01 12:57:19,NaN
4,593584377218473984,@frogtosser Thank you Becca.,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0.000000e+00,14249445,frogtosser,0.000000e+00,\N,\N,NaN,NaN,0.0,0,0,NaN,NaN,\N,0.0,0,1,Thu Apr 30 01:15:18 +0000 2015,2015-04-30 03:15:18,2015-05-01 12:57:19,2015-05-01 12:57:19,NaN


In [11]:
# Genuine accounts in test set 1.
# gen_test1 = gen[gen['test_set_1'] == 1]

# Social spambots in test set 1.
ss1_test_1 = ss1[ss1['test_set_1'] == 1]


# Tweets for genuine accounts in test set 1.
# gen_tweets_test1 = gen_tweets[gen_tweets['user_id'].isin(gen_test1['id'])]

# Tweets for spambot accounts in test set 1.
ss1_tweets_test1 = ss1_tweets[ss1_tweets['user_id'].isin(ss1_test_1['id'])]

In [27]:
print(ss1.count)
print(ss1_test_1.count)

<bound method DataFrame.count of              id                 name  ...              updated  test_set_1
0      24858289       Davide Bertoli  ...  2016-03-15 14:12:22           1
1      33212890       Elisa D'Ospina  ...  2016-03-15 14:17:13           1
2      39773427      Donato Vincenzo  ...  2016-03-15 14:16:44           1
3      57007623   Rafiela Morales L.  ...  2016-03-15 14:18:54           1
4      63258466         § h a † u r♄  ...  2016-03-15 14:17:05           1
..          ...                  ...  ...                  ...         ...
986  2286273170              falomar  ...  2016-03-15 14:14:19           1
987  2287318956         Marco Nuovoi  ...  2016-03-15 14:14:54           1
988  2290264806                 Enzo  ...  2016-03-15 14:12:11           1
989  2398910547      Southern Fields  ...  2016-03-15 14:16:12           1
990  2525273432  CHIARA PERISSINOTTO  ...  2016-03-15 14:19:44           1

[991 rows x 41 columns]>


NameError: ignored

In [28]:
print(gen.count)
print(gen_test1.count)

<bound method DataFrame.count of               id               name  ... test_set_1  test_set_2
0     1502026416    TASUKU HAYAKAWA  ...          0           0
1     2492782375              ro_or  ...          0           0
2      293212315           bearclaw  ...          0           0
3      191839658  pocahontas farida  ...          0           0
4     3020965143           Ms Kathy  ...          0           0
...          ...                ...  ...        ...         ...
3469   205218909         Alejandro   ...          0           0
3470  2874966164    Zubair Niazi Ⓜ️  ...          0           0
3471  2980901837            Zuhazuu  ...          0           0
3472   121122678            zveljka  ...          0           0
3473  2910276853       // karime //  ...          1           1

[3474 rows x 42 columns]>


NameError: ignored

In [34]:
print(ss1_tweets.count)
print(ss1_tweets_test1.size)

<bound method DataFrame.count of                          id  ...              updated
0        532627591686275072  ...  2014-11-12 21:44:09
1        532624255058706432  ...  2014-11-12 21:44:09
2        532513524460052480  ...  2014-11-12 21:44:09
3        532297646669852672  ...  2014-11-12 21:44:09
4        532295960807100416  ...  2014-11-12 21:44:09
...                     ...  ...                  ...
1610029  470953862489468928  ...  2014-11-14 01:40:34
1610030  470953620155158528  ...  2014-11-14 01:40:34
1610031  470953572868562945  ...  2014-11-14 01:40:34
1610032  470953492358901760  ...  2014-11-14 01:40:34
1610033  470953465309822977  ...  2014-11-14 01:40:34

[1610034 rows x 25 columns]>


NameError: ignored

In [11]:
print(gen_tweets.size)
print(gen_tweets_test1.size)

27000


NameError: ignored

In [29]:
gen_tweets.head()

,593932392663912449,"RT @morningJewshow: Speaking about Jews and comedy tonight at Temple Emanu-El in San Francisco. In other words, my High Holidays.","<a href=\http://tapbots.com/tweetbot\"" rel=\""nofollow\"">Tweetbot for iΟS</a>""",678033,Unnamed: 4,0,0.1,Unnamed: 7,593932168524533760,\N,\N.1,Unnamed: 11,Unnamed: 12,1,0.2,0.3,Unnamed: 16,Unnamed: 17,\N.2,0.4,0.5,1.1,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19.1
0,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=\http://twitter.com\"" rel=\""nofollow\""...",678033,NaN,0,0,NaN,0,\N,\N,NaN,NaN,0,0,0,NaN,NaN,\N,0,0,0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19


In [26]:
gen_tweets.rename(columns = {'Unnamed: 0':'user_id'}, inplace = True)

In [17]:
ss1_tweets_test1.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,num_retweets,num_replies,num_tweets,DNA
0,532627591686275072,I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:14:48 +0000 2014,2014-11-12 21:14:48,2014-11-12 21:44:09,2014-11-12 21:44:09,0,0,1,A
1,532624255058706432,http://t.co/HyI5EQKz6Q,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:01:32 +0000 2014,2014-11-12 21:01:32,2014-11-12 21:44:09,2014-11-12 21:44:09,0,0,1,A
2,532513524460052480,"Tutti a tavola, con il filetto di baccalà. htt...","<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Wed Nov 12 12:41:32 +0000 2014,2014-11-12 13:41:32,2014-11-12 21:44:09,2014-11-12 21:44:09,0,0,1,A
3,532297646669852672,http://t.co/NAHQ4l2pUy,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:23:43 +0000 2014,2014-11-11 23:23:43,2014-11-12 21:44:09,2014-11-12 21:44:09,0,0,1,A
4,532295960807100416,Gold - Spandau Ballet http://t.co/o8ZJHt7Neu,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:17:01 +0000 2014,2014-11-11 23:17:01,2014-11-12 21:44:09,2014-11-12 21:44:09,0,0,1,A


In [19]:
def create_dna_from_tweets(tweets_df):
    '''For each user id in tweets_df return a digital DNA string based on posting behaviour.'''
    
    # Add columns for counts of tweets, replies and retweets.
    tweets_df['num_retweets'] = np.where(tweets_df['retweeted_status_id'] == 0, 0, 1)
    tweets_df['num_replies'] = np.where(tweets_df['in_reply_to_status_id'] == 0, 0, 1)
    tweets_df['num_tweets'] = np.where((tweets_df['num_retweets'] == 0) & (tweets_df['num_replies'] == 0), 1, 0)

    # DNA alphabet for tweet (A), retweet (C) and reply (T).
    tweets = tweets_df['num_tweets'] == 1
    retweets = tweets_df['num_retweets'] == 1
    replies = tweets_df['num_replies'] == 1


    tweets_df.loc[:, 'DNA'] = np.where(retweets, 'C', np.where(replies, 'T', 'A'))

    # Sort tweets by timestamp.
    tweets_df = tweets_df[['user_id', 'timestamp', 'DNA']]
    tweets_df = tweets_df.sort_values(by=['timestamp'])

    # Create digital DNA string for account.
    dna = tweets_df.groupby(by=['user_id'])['DNA'].agg(lambda x: ''.join(x))
    
    return dna

In [23]:
def compress_dna_df(dna):
    '''Return a dataframe with compression facts for a series of dna.'''

    # Convert DNA in string object to bytes object.
    dna_bytes = dna.apply(lambda s: s.encode('utf-8'))

    # Run compression on each DNA string in the sample.
    dna_compressed = dna_bytes.apply(lambda b: zlib.compress(b))

    # Create dataframe with compression facts.
    dna_df = pd.DataFrame({'dna': dna,
                           'original_dna_size': dna_bytes.apply(sys.getsizeof), 
                           'compressed_dna_size': dna_compressed.apply(sys.getsizeof)})
    
    dna_df['compression_ratio'] = dna_df['original_dna_size'] / dna_df['compressed_dna_size']
    
    return dna_df

In [27]:
# DNA for genuine accounts in test set 1.
# gen_dna_test1 = create_dna_from_tweets(gen_tweets_test1)

# DNA for spambots in test set 1.
ss1_dna_test1 = create_dna_from_tweets(ss1_tweets_test1)


# DNA string compression for genuine accounts in test set 1.
# gen_dna_test1 = compress_dna_df(gen_dna_test1)

# DNA string compression for spambots in test set 1.
ss1_dna_compressed_test1 = compress_dna_df(ss1_dna_test1)
ss1_dna_compressed_test1['label'] = 1



0           True
1           True
2           True
3           True
4           True
           ...  
1610029     True
1610030    False
1610031    False
1610032    False
1610033    False
Name: num_tweets, Length: 1610034, dtype: bool


In [22]:
ss1_dna_test1.head()

user_id
24858289    AAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
33212890    TCACCCACAACCAAAAAAACCAAAAAAAATATTAAAATTTTTTAAT...
39773427    AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
57007623    AAAAAAAAAAAAAAAAAAAAAAAAAATAAAAAAACAAAAAAAAAAA...
63258466    ATAAAAAAAAAAAACAAAATTATATTTTTATTATATTTTTATTTAA...
Name: DNA, dtype: object

In [28]:
ss1_dna_compressed_test1.head()

,dna,original_dna_size,compressed_dna_size,compression_ratio,label
user_id,,,,,
24858289,AAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,2551,98,26.030612,1
33212890,TCACCCACAACCAAAAAAACCAAAAAAAATATTAAAATTTTTTAAT...,3237,766,4.225849,1
39773427,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,3249,273,11.901099,1
57007623,AAAAAAAAAAAAAAAAAAAAAAAAAATAAAAAAACAAAAAAAAAAA...,3264,626,5.214058,1
63258466,ATAAAAAAAAAAAACAAAATTATATTTTTATTATATTTTTATTTAA...,3234,701,4.613409,1
